In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import pandas as pd

##### You should specify the path to the chromedriver that is located in your computer and assign it to the path variable below.

In [2]:
url = 'https://en.bidfax.info/'
path = '/Users/areg/.wdm/drivers/chromedriver/mac64/108.0.5359/chromedriver'
# You should specify the path of the chromedriver in the brackets 
# that are in front of path variable above
driver = webdriver.Chrome(path)
driver.get(url)

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_37759/3681283350.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


#### After calling select_make() function it will ask you to input a make of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [3]:
def select_make():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    make = str(input('Please write the make you want to select:  '))
    def search_for_make(make):
        try:
            dropdown[0].click()
        except:
            pass
        try:
            make_ = driver.find_element(By.LINK_TEXT, make)
            make_.click()
            print(f"{make} make is selected successfully!")
        except:
            print(f'{make} make does not exist in this list. Please try again.')
            return select_make()
    search_for_make(make)
    time.sleep(1)

#### After calling select_model() fucntion it will ask you to input a model of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [4]:
def select_model():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    model = str(input('Please write the model you want to select:  '))
    def search_for_model(model):
        try:
            dropdown[1].click()
        except:
            pass
        try:
            model_ = driver.find_element(By.LINK_TEXT, model)
            model_.click()
            print(f"{model} model is selected successfully!")
        except:
            print(f'{model} model does not exist in this list. Please try again.')
            return select_model()
    search_for_model(model)  
    time.sleep(1)

#### After calling select_year_start() function below it will ask you to input the starting year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one. Call the select_year_start function after calling select_make() and select_model() functions

In [5]:
def select_year_start():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))
    scrape_years()
    start = str(input("Type in the starting year of the car:  "))
    def year_start(start):   
        year_start = years[0]
        try:
            year_start = year_start.find_elements(By.XPATH, ("./option"))
            for year in year_start:
                if year.text == start:
                    print(f"{start} year is selected successfully!")
                    return year.click()
            raise print(f"{start} year does not exist. Please try again.")
        except:
            return select_year_start()
    year_start(start)
    time.sleep(1)

#### After calling select_year_to() function it will ask you to input the ending year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one.  Call the select_year_start function after calling select_make() and select_model() functions

In [6]:
def select_year_to():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))    
    scrape_years()
    to = str(input("Type in the ending year of the car:  "))
    def year_end(year_to):   
        year_to = years[1]
        try:
            year_to = year_to.find_elements(By.XPATH, ("./option"))
            for year in year_to:
                if year.text == to:
                    print(f"{to} year is selected successfully!")
                    return year.click()
            raise print(f"{to} year does not exist. Please try again.")
        except:
            return select_year_to()
    year_end(to)
    time.sleep(1)
    

In [3]:
def scrape_page():
    global data
        
    lot_number = []; date_of_sale = []; year = []; vin = []
    condition = []; engine = []; mileage = []; documents = []
    location = []; primary_damage = []; secondary_damage = []
    estimated_retail_value = [];estimated_repair_cost = []
    transmission = []; body_color = [];drive = []; fuel = []
    keys = []; notes = []; avg_price = []; vehicles = []
    price = []; market = []; seller = []
    
    def scrape_data():
        global info, info_p, auction, columns, data
        info = driver.find_elements(By.XPATH, '//span[@class = "blackfont"]')
        info_p = driver.find_element(By.XPATH, '//span[@class = "prices"]')
        auction = driver.find_element(By.XPATH, '//p[@class ="short-story"]/child::span')   


        data = [vehicles, avg_price, lot_number, date_of_sale, 
                year, vin, condition, engine, mileage,  
                documents, location, primary_damage, 
                secondary_damage, estimated_retail_value, 
                estimated_repair_cost, transmission, body_color,
                drive, fuel, keys, notes, seller, price, market] 
        columns = ['vehicles', 'avg_price', 'lot_number', 'date_of_sale', 
                   'year', 'vin', 'condition', 'engine', 'mileage', 
                   'documents', 'location', 'primary_damage', 'secondary_damage', 
                   'estimated_retail_value', 'estimated_repair_cost', 
                   'transmission', 'body_color','drive', 'fuel', 'keys', 
                   'notes','seller', 'price', 'market']   
    
    def adding_data_to_lists():
        scrape_data()
        price.append(info_p.text)
        market.append(auction.text)
        count_ = 0
        if auction.text == ' Copart ':
            data[21].append(info[9].text)
            for i in (info[:9]+info[10:22]):
                data[count_].append(i.text)
                count_ += 1  
        else:
            seller_ = driver.find_element(By.XPATH, '//p[@class = "iaaiseller"]')
            data[21].append(seller_.text)
            for i in info[:21]:
                data[count_].append(i.text)
                count_ += 1
    def go_in_car():
        cars = driver.find_elements(By.XPATH,'//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]')
        for i in range(len(cars)):
            a = (driver.find_elements(By.XPATH, '//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]/parent::p/preceding-sibling::a'))
            a[i].click()
            time.sleep(1)
            adding_data_to_lists()
            driver.current_window_handle
            driver.back()
            time.sleep(1)
    go_in_car()
scrape_page()


In [13]:
dict_ = {}
for i, j in zip(columns, data):
    dict_[i] = j
df = pd.DataFrame(dict_)
df

,vehicles,avg_price,lot_number,date_of_sale,year,vin,condition,engine,mileage,documents,...,estimated_repair_cost,transmission,body_color,drive,fuel,keys,notes,seller,price,market
0,1162,$18636,73686162,06.02.2023,2021,5YJ3E1EA4MF850355,Run And Drive,Not specified or electro,22195 miles (Actual),Fl - Cert Of Title Slvg Rebuildable,...,33471,Automatic,White,Rear-Wheel Drive,Electric,Yes,Not specified,Not specified,16400,Copart
1,7,$34007,35349174,06.02.2023,2010,ZFF65LJA9A0168453,Stationary,4.3L DI V8 FLAT PROPELLER,18799 miles (Actual),SALVAGE (New Jersey),...,131981,Automatic,Yellow,Rear Wheel Drive,Gasoline,Present,Not specified,Seller: New Jersey Manufacturing Group,39100,IAAI
2,5,$90210,72586872,03.02.2023,2019,SBM13FAA4KW006350,Enhanced Vehicles,3.8L 8,27298 miles (Actual),Tx - Salvage Vehicle Title,...,191120,Automatic,Red,Rear-Wheel Drive,Gas,Yes,Not specified,Not specified,42000,Copart
3,1,$115000,68488692,03.02.2023,2012,ZHWUC1ZD0CLA00212,Enhanced Vehicles,6.5L 12,0 miles (Not Actual),Oh - Cert Of Title-Salvage,...,149664,Manual,White,All Wheel Drive,Gas,No,Not specified,Not specified,94500,Copart
4,,$,62770192,03.02.2023,2012,ZFF68NHA0C0188795,Enhanced Vehicles,4.5L 8,0 miles (Not Actual),Fl - Certificate Of Destruction,...,276196,Automatic,Burgundy,Rear-Wheel Drive,Gas,Yes,Not specified,Not specified,92500,Copart
